In [1]:
import scipy
import cartoframes
import pandas as pd
import pysal
import geopandas as gpd
import networkx as nx
import numpy as np

In [2]:
cc = cartoframes.CartoContext()

In [3]:
find_nearest_query = '''
SELECT
  roads.cartodb_id as cartodb_id,
  count(collisions.cartodb_id) as collision_count,
  sum(collisions.number_of_persons_injured) as sum_injuries,
  sum(collisions.number_of_persons_killed) as sum_fatalities,
  sum(collisions.number_of_cyclist_injured) as sum_cyclist_injured,
  sum(collisions.number_of_cyclist_killed) as sum_cyclist_killed,
  sum(collisions.number_of_motorist_injured) as sum_motorists_injured,
  sum(collisions.number_of_motorist_killed) as sum_motorists_killed,
  sum(collisions.number_of_pedestrians_injured) as sum_pedestrian_injured,
  sum(collisions.number_of_pedestrians_killed) as sum_pedestrian_killed 
FROM
  (SELECT cartodb_id, the_geom, the_geom_webmercator, number_of_cyclist_injured,
        number_of_cyclist_killed,
        number_of_motorist_injured,
        number_of_motorist_killed,
        number_of_pedestrians_injured,
        number_of_pedestrians_killed,
        number_of_persons_injured,
        number_of_persons_killed
   FROM nypd_collisions_2017) AS collisions
CROSS JOIN LATERAL
  (SELECT cartodb_id, the_geom, the_geom_webmercator
   FROM {road_dataset} as roads
   ORDER BY collisions.the_geom_webmercator <-> the_geom_webmercator
   LIMIT 1) AS roads
GROUP BY roads.cartodb_id
'''

In [5]:
tomtom_collisions = cc.query(find_nearest_query.format(road_dataset='joined'), table_name='tomtom_collisions', decode_geom=True)

Table successfully written to CARTO: https://michellemho-carto.carto.com/dataset/tomtom_collisions


In [4]:
tomtom_collisions = cc.read('tomtom_collisions')

In [5]:
road_segments = cc.read('joined', decode_geom=True)
road_segments.reset_index(inplace=True)

In [10]:
road_segments.head()

,cartodb_id,ada,backrd,carriage,charge,constatus,dynspeed,f_bp,f_elev,f_jnctid,...,we_23,we_24,we_3,we_4,we_5,we_6,we_7,we_8,we_9,geometry
0,3043,0,0,None,None,None,0,0,0,2147483647,...,0,0,0,0,0,0,0,1,4,"(LINESTRING (-74.168707 40.5768001, -74.168853..."
1,36,0,0,None,None,None,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,0,"(LINESTRING (-73.546876 41.980526, -73.5472709..."
2,130,1,0,None,None,None,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,0,"(LINESTRING (-73.232291 40.630219, -73.231802 ..."
3,282,0,5,None,None,None,0,0,0,2147483647,...,5,0,0,0,0,0,0,2,3,"(LINESTRING (-74.21202099999999 40.551974, -74..."
4,470,1,0,None,None,None,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,0,"(LINESTRING (-74.1673138 40.5788879, -74.16710..."


In [6]:
tomtom_collisions_joined = road_segments.join(tomtom_collisions, on='cartodb_id', how='left', lsuffix='_left')

In [7]:
tomtom_collisions_joined.shape

(947079, 113)

In [8]:
tomtom_collisions_joined.collision_count.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_injuries.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_fatalities.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_cyclist_injured.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_cyclist_killed.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_motorists_injured.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_motorists_killed.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_pedestrian_injured.fillna(value=0, inplace=True)
tomtom_collisions_joined.sum_pedestrian_killed.fillna(value=0, inplace=True)

In [12]:
cc.write(tomtom_collisions_joined, 'tomtom_collisions_joined')

Uploading in batches:   0%|          | 0/2 [00:00<?, ?it/s]/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
Uploading in batches: 100%|██████████| 2/2 [09:47<00:00, 293.84s/it]


NameError: name 'warn' is not defined

In [ ]:
# matches the number of rows in the nypd collision dataset
collision_join.collision_count.sum()